In [1]:
%load_ext autoreload
%autoreload 2

import h5py
import glob
import numpy as np
import nibabel as nib

import os.path

import sys
assert sys.version_info.major == 3, 'Not running on Python 3'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
ds_path = "/home/imag2/IMAG2_DL/KDCompression/Dataset/ds.hdf5"

img_dir = "/home/imag2/IMAG2_DL/APMRI-DNN/Dataset/All"
teacher_dir = "/home/imag2/IMAG2_DL/KDCompression/Dataset/Teacher/"

imgs = sorted(glob.glob(os.path.join(img_dir, '*[tT]2*.nii.gz')))
masks = sorted(glob.glob(os.path.join(img_dir, '*[mM]ask*.nii.gz')))
labels = sorted(glob.glob(os.path.join(img_dir, '*[sS]egmentation*.nii.gz')))
teacher_labels = sorted(glob.glob(os.path.join(teacher_dir, '*[tT]eacher*.nii.gz')))

In [3]:
def load_nii(f_name):
    img = nib.load(f_name)
    canonical_img = nib.as_closest_canonical(img)
    return canonical_img.get_fdata()

In [ ]:
img = []
mask = []
label = []
teacher_label = []
for i, im in enumerate(imgs[:2]):
    im =load_nii(im)
    #mask.append(load_nii(m))
    #label.append(load_nii(l))
    #teacher_label.append(load_nii(t))


with h5py.File(ds_path, "a") as f:
    student = f.create_group("student")
    teacher = f.create_group("teacher")
    imgs = student.create_group("img")
    masks = student.create_group("mask")
    label = student.create_group("label")

In [ ]:
with h5py.File(ds_path, 'a', libver='latest') as f:
    for idx, arr in enumerate(imgs):
        arr = load_nii(arr)
        dset = f.create_dataset(str(idx), shape=arr.shape, data=arr, chunks=arr.shape,
                                compression='gzip', compression_opts=9)
        